### EBirds API 

Get data from Ebirds API for the count of birds in the sky at a given location from year 2000 till present date. Calculate the distance from Denver International Airport and filter the list to less than 10 miles as we are **assuming** that airplanes travel approximately 10 miles to climb a height of 2000 Ft altitude. Obviously, this cannot be generalized across all the flights as the climb various from smaller airframes to larger  but this could an educated guess based on our analysis.

In [4]:
from geopy.distance import geodesic
import pandas as pd
import json
from ebird.api import Client
from pandas.io.json import json_normalize 
from datetime import datetime
from datetime import datetime, timedelta

#### Set up API

In [5]:
# Declare all static values for calling the API

api_key = 'ldto4uofjnf7'
locale = 'en'
region_codes = ['US-CO-031','US-CO-001','US-CO-005', 'US-CO-059']
startDate = datetime.strptime('2018/03/01', "%Y/%m/%d")
endDate = datetime.strptime('2019/12/31', '%Y/%m/%d')
client = Client(api_key, locale)
filename = 'birdCount_DEN_1.csv'


# Latitude and Longitude for Denver International Airport
dia = (39.861698150635, -104.672996521)

# Declare an empty dataframe to store all the records
df = pd.DataFrame(columns = ['obsId', 'locId', 'locName', 'obsDt', 'lat', 'lng', 'subnational2Code', 'subnational2Name', 'comName', 'sciName', 'speciesCode', 'howMany'])

#### Call API with credentials and other required headers and save data to file

In [6]:

# Call the API till startdate and enddate match
while(startDate != endDate):
    
    # Iterate through all the regions around DIA Airport
    for code in region_codes:
        
        # Call to API
        response = client.get_historic_observations(code, startDate)
        if len(response) > 0:
            # Convert the response JSON to a Pandas dataframe
            df = pd.DataFrame(response)
            
            # If howMany Column doesn't exist then create that and fill with 0's
            if 'howMany' not in df.columns:
                df['howMany'] = 0
                
            # Get only the required fields into the dataframe
            df = df[['obsId', 'locId', 'locName', 'obsDt', 'lat', 'lng', 'subnational2Code', 'subnational2Name', 'comName', 'sciName', 'speciesCode', 'howMany']]
        else:
            print("No data received from the API for", startDate, code)

        # Create a new column called milesFromAirport
        if 'milesFromAirport' not in df:
            df['milesFromAirport'] = 0.00        
            
        # Iterate through all the rows and calculate the distance from 
        # DIA airport coordinates provided in the ICAO Codes
        for index, row in df.iterrows():
            
            # Create Location object
            location = (row['lat'], row['lng'])

            # Calculate distance
            df.loc[index, 'milesFromAirport'] = geodesic(dia, location).miles
            
        # Filter data in the dataframe to less than 20 miles from DIA
        df = df[df['milesFromAirport'] <= 12]
        
        #Save data to csv file
        with open(filename, 'a') as f:
            df.to_csv(f, mode='a', header=False)
            
    # Increament startDate to next day
    startDate = startDate + timedelta(days=1)


#### Aggregate data based on Observation date

In [7]:
# reading csv file  
bc = pd.read_csv("birdCount_DEN.csv", header=0)

for index, row in bc.iterrows():
    bc.loc[index, 'observationDate'] = datetime.strptime(row['obsDt'].split(' ')[0], "%m/%d/%y")

In [9]:
#Save data to csv file
bc.to_csv(r'birdCount_Modified_DEN.csv', index = None, header=True)

In [10]:
bcgroup = bc.groupby('observationDate').agg({'howMany':'sum'}).reset_index().rename(columns={'howMany':'birdCount'})

In [12]:
#Save data to csv file
bcgroup.to_csv(r'birdCount_Aggregated_DEN.csv', index = None, header=True)